In [1]:
#pip install -U git+https://github.com/QuantFreedom1022/quantfreedom@dev

In [3]:
#pip install vectorbt

In [2]:
#installation of Yahoo Finance 
!pip install yfinance


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached yfinance-0.2.14-py2.py3-none-any.whl (59 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached lxml-4.9.2-cp310-cp310-win_amd64.whl (3.8 MB)
  Using cached frozendict-2.3.6-cp310-cp310-win_amd64.whl (34 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)


In [3]:
### Importing QuantFreedom and all requirements for strategy creation

import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

from quantfreedom.nax_qf.eval_nax import eval_is_above

import quantfreedom as qf
#import vectorbt as vbt
import yfinance as yf
import numpy as np
import pandas as pd
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

import plotly.express as px
import plotly.graph_objects as go

import talib
from talib.abstract import Function
from talib import abstract

# ^ - importing in next cell
# ^^^ from quantfreedom.backtester.indicators.talib_ind import from_talib, talib_func_list_website_link, talib_ind_info
# ^^^ 
from quantfreedom.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
from quantfreedom._typing import (
    pdFrame, pdSeries)


#!!!!!!!!!! ############################ if I choose kernel Python 3.10, I got error on 8 cell. If I choose qfFree, I got error on cell 8
# it's start to happen when I'm uncomment this row: from quantfreedom.backtester.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
"""
Trying to fix it: 1) Delete and install extensions again: jupyter, python, Pylance, Python Environment Manager DONE! Not helped.

2) Removing all venv by a) jupyter kernelspec list b) than: jupyter kernelspec remove <kernel_name>

3) Uninstall Python in Windows and install again
"""


'\nTrying to fix it: 1) Delete and install extensions again: jupyter, python, Pylance, Python Environment Manager DONE! Not helped.\n\n2) Removing all venv by a) jupyter kernelspec list b) than: jupyter kernelspec remove <kernel_name>\n\n3) Uninstall Python in Windows and install again\n'

In [4]:
####### Importing all QuantFreedom backtesting stuff #######

# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

# ^ - already imported in previous cell
# ^^^ import plotly.express as px
# ^^^ import plotly.graph_objects as go

# ^ - already imported in previous cell
# ^^^ import pandas as pd
# ^^^ import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.nb.simulate import backtest_df_array_only
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below

# np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

# pd.options.display.float_format = '{:,.2f}'.format

In [5]:
# Fetch historical data

# determine variable 'symbol' for fetching many tickets simultaneously (at same time)
# symbol = [
#     'BTC-USD', 'ETH-USD', 'XRP-USD', 'BCH-USD', 'LTC-USD'#,
# ]
# """    'BNB-USD', 'EOS-USD', 'XLM-USD', 'XMR-USD', 'ADA-USD',
#     'QBIT-USD', 'TSLA', 'AMZN', 'AAPL', 'DGTX-USD', 'BTG-USD', 
#     'LINK-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD',
#     'TON-USD', 'OKB-USD', 'GLD', 'GIS', 'PG', 'MRK', 'AMT', 'BMY',
#     'CVX', 'S', 'CVS', 'HD', 'PUK'
# #]
# """


symbol = ['BTC-USD']

# "Attach" dataset prices (dataframe) to variable ohlcv and at the same time download dataset from Yahoo Finance 
ohlcv = yf.download(symbol, interval="5m", period="60d")

# "Attach" only closing prices from dataset to variable 'price'
prices = ohlcv['Close']

# "Attach" only High prices from dataset to variable 'high_price'
high_price = ohlcv['High']

# Same for low
low_price = ohlcv['Low']

[*********************100%***********************]  1 of 1 completed


In [6]:
#Renaming columns for TA-Lib
ohlcv.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Adj Close': 'close', 'Volume': 'volume'}, inplace=True)

In [7]:
ohlcv.head(5)

,open,high,low,Close,close,volume
Datetime,,,,,,
2023-02-03 00:00:00+00:00,"23,469.41","23,550.28","23,469.41","23,550.28","23,550.28",0
2023-02-03 00:05:00+00:00,"23,553.86","23,556.62","23,548.30","23,556.62","23,556.62",0
2023-02-03 00:10:00+00:00,"23,562.35","23,575.52","23,562.35","23,574.48","23,574.48",10799104
2023-02-03 00:15:00+00:00,"23,579.59","23,581.61","23,570.70","23,570.70","23,570.70",9125888
2023-02-03 00:20:00+00:00,"23,567.41","23,580.04","23,566.72","23,578.90","23,578.90",0


In [8]:
prices.head(5)

Datetime
2023-02-03 00:00:00+00:00   23,550.28
2023-02-03 00:05:00+00:00   23,556.62
2023-02-03 00:10:00+00:00   23,574.48
2023-02-03 00:15:00+00:00   23,570.70
2023-02-03 00:20:00+00:00   23,578.90
Name: Close, dtype: float64

In [9]:
 
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=30,
)


 
# talib.abstract.RSI(prices)


In [10]:
# Define Divergence
def divergence(
    # using price set as starting point to calculate divergence
    price_set,
    # using last 14 bars/days period window for RSI indicator. This is just a template and can be changed when we will execute .run function in next stage.
    rsi_window,
    # using last 1 bar/day period window for fast Moving Average indicator. This is just a template and can be changed when we will execute .run function in next stage.
    fast_ma_window,

    # using last 50 bars/days period window for fast Moving Average indicator.  This is just a template and can be changed when we will execute .run function in next stage.
        slow_ma_window,):

    slow_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=slow_ma_window,
    )
    fast_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=fast_ma_window,
    )
    rsi_indicator = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=rsi_window,
    ).squeeze()

# Calculate the Divergence indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics
    bullish_divergence = ((price_set.diff() < 0) & (rsi_indicator.diff() > 0)).to_numpy()

    bearish_divergence = ((price_set.diff() < 0) &
                          (rsi_indicator.diff() < 0)).to_numpy()

    # Determine UpTrend (fast MA above slow MA)
    trend_up = eval_is_above(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )
    trend_down = eval_is_below(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )

    # Convert everything to Numpy array
    trend_up = trend_up.to_numpy().flatten()
    trend_down = trend_down.to_numpy().flatten()

    # Combine all previous indicators into the one
    # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
    indicator_buy = np.logical_and(bullish_divergence == True,trend_up == True)
    indicator_buy = np.where(indicator_buy == True, 1, 0)
    indicator_sell = np.logical_and(bearish_divergence == True,trend_down == True)
    indicator = np.where(indicator_sell == True, -1, indicator_buy)


    return indicator


In [11]:
print(type(divergence))


price_set = ohlcv['close']
rsi_window = 14
fast_ma_window = 1
slow_ma_window = 50

# Call the function and print the value of indicator
indicator = divergence(price_set, rsi_window, fast_ma_window, slow_ma_window)
print(indicator)

<class 'function'>
[0 0 0 ... 0 0 0]


In [12]:
# Just for testing purposes - can be deleten

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(1),
)


rsi_eval.head(5)
 


RSI_timeperiod,14
RSI_is_below,0
Datetime,
2023-02-03 00:00:00+00:00,False
2023-02-03 00:05:00+00:00,False
2023-02-03 00:10:00+00:00,False
2023-02-03 00:15:00+00:00,False
2023-02-03 00:20:00+00:00,False


In [13]:
# Define backtester

strat_array, settings_array = backtest_df_array_only(
    open_prices=ohlcv.open.values,
    high_prices=ohlcv.high.values,
    low_prices=ohlcv.low.values,
    close_prices=ohlcv.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.02,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1,6.1,1),
    risk_rewards=np.arange(1,10.1,1),
    size_pct=1.,
    gains_pct_filter=100,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1,6.1,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,5.1,1),
    tsl_when_pct_from_avg_entry=np.arange(1,5.1,1),
)

In [ ]:
# Evaluators and backtesting


In [14]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=1,
)

talib.SMA(atr_ind.values.flatten(), timeperiod=50)

array([nan, nan, nan, ..., 42.21, 42.23, 41.36])

In [ ]:
"""
pf = vbt.Portfolio.from_signals(ohlcv, divergence(indicator_buy), divergence(indicator_sell))

# Output
print(pf.total_return())
print(pf.stats())
"""

In [15]:
divergence(
    price_set=ohlcv.close,
    rsi_window=15,
    fast_ma_window=10,
    slow_ma_window=40,
)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=1,
)

In [ ]:
atr_ind

In [ ]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50)


In [ ]:
(1000 / ohlcv.close.values.flatten())

In [ ]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50) * (1000 / ohlcv.close.values.flatten())

In [ ]:
fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
#printing tpe of bearish_divergence 
print("printing tpe of bearish_divergence dataset:")
print(type(bearish_divergence))
# Get slow Moving Average (MA) of the prices

atr = vbt.ATR.run(high_price, low_price, price, atr_window)
#atr = atr.atr.to_numpy()

atr = atr.atr.to_numpy()
##print(type(atr))

# Calculate adaptive/dynamically changing value based on ATR and current price 
atr_results = atr * atr_scaler / price #* rsi_window

slow_ma = vbt.MA.run(atr_results, slow_ma_window, short_name='slow')
print("printing tpe of slow_ma dataset:")
print(type(slow_ma))

In [ ]:
fig = go.Figure()
fig.add_scatter(
    x=atr_ind.index.to_list(),
    y=ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='red'),
    name='ATR'

)
fig.add_scatter(
    x=rsi_ind.index.to_list(),
    y=talib.SMA(atr_ind.values.flatten(), timeperiod=100) * 1000 / ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='blue'),
    name='SMA'
)
fig.show()


In [ ]:
atr_ind.values.flatten()

In [ ]:
atr_ind

In [ ]:
atr_ma = from_talib(
    func_name='MA',
    df_prices=None,
    cart_product=False,
    combos=False,
    timeperiod=50,
    price=atr_ind.values.flatten()
)

In [ ]:
talib_ind_info('stoch')

In [ ]:
close = np.random.random(100)
open = np.random.random(100)
high = np.random.random(100)

In [ ]:
Function('sma')(np.random.random(10000), 50)

In [ ]:
_, hey = talib.STOCH(close, open, high, *(10,15))
hey